# IRS `net` migration at the county level: 2011-2020

#### Import Python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import altair as alt
import altair_grid as altgrid
import numpy as np
import us

In [3]:
alt.themes.register("grid", altgrid.theme)
alt.themes.enable("grid")

ThemeRegistry.enable('grid')

In [4]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

## Read [migration data](https://www.irs.gov/statistics/soi-tax-stats-data-by-geographic-area) from IRS. Processed in `00` and `01` notebooks. 

In [5]:
inflow = pd.read_csv(
    "data/processed/irs_migration_arrived_to_mean.csv", dtype={"destination_fips": str}
)

In [6]:
inflow.head()

,destination_fips,destination_county_name,destination_state_abbr,returns
0,06037,Los Angeles County,CA,113090.0
1,48201,Harris County,TX,76368.0
2,04013,Maricopa County,AZ,74646.0
3,17031,Cook County,IL,69922.0
4,36061,New York County,NY,62614.0


In [7]:
outflow = pd.read_csv(
    "data/processed/irs_migration_left_from_mean.csv", dtype={"origin_fips": str}
)

In [8]:
outflow.head()

,origin_fips,origin_county_name,origin_state_abbr,returns
0,06037,Los Angeles County,CA,140088.0
1,17031,Cook County,IL,90530.0
2,48201,Harris County,TX,79482.0
3,36061,New York County,NY,76950.0
4,36047,Kings County,NY,66414.0


In [9]:
src = (
    pd.merge(outflow, inflow, left_on="origin_fips", right_on="destination_fips")
    .rename(columns={"returns_x": "left", "returns_y": "arrived"})
    .copy()
)

In [10]:
src["net"] = src["arrived"] - src["left"]

In [11]:
df = (
    src[
        [
            "origin_fips",
            "origin_county_name",
            "origin_state_abbr",
            "left",
            "arrived",
            "net",
        ]
    ]
    .rename(
        columns={
            "origin_fips": "fips",
            "origin_county_name": "county_name",
            "origin_state_abbr": "state_abbr",
        }
    )
    .sort_values("net", ascending=False)
    .copy()
)

In [14]:
df.sort_values("net", ascending=False).head()

,fips,county_name,state_abbr,left,arrived,net
7,04013,Maricopa County,AZ,58957.0,74646.0,15689.0
28,32003,Clark County,NV,32443.0,43582.0,11139.0
13,48453,Travis County,TX,47243.0,55220.0,7977.0
19,06065,Riverside County,CA,41831.0,48614.0,6783.0
101,12071,Lee County,FL,12614.0,18715.0,6101.0


---

## Exports

In [13]:
df.to_csv("data/processed/irs_migration_average_net.csv", index=False)